# Recitation 3

Topics: ASTs, higher order functions, pattern functions

## SQL Pt 1

### Introduction to SQL

SQL (**S**tructured **Q**uerying **L**anguage) is a domain-specific language used to manage and query data. Data is stored in tables. For example, here is a table called **customer** that stores my company's customer records.

| id | name | address | city | postal_code | country |
| --- | --- | --- | --- | --- | --- |
| 1 | Maria Anders | Obere Str. 57 | Berlin | 12209 | Germany |
| 2 | Ana Trujillo | Avda. de la Constitución 2222 | Mexico City | 05021 | Mexico |
| 3 | Antonio Moreno | Mataderos 2312 |	Mexico City | 05023	| Mexico |
| 4 | Thomas Hardy | 120 Hanover Sq. | London | WA1 1DP | UK |
| 5 | Christina Berglund | Berguvsvägen 8 | Luleå | S-958 22 | Sweden |

Each table is defined by its name (**customer**) and its columns (**id**, **name**, **address**, **city**, etc). Each table is populated by rows, each of which represents a data record and maps each of the table's columns to values.

Given a database consisting of one or more tables, SQL makes it easy for me to ask questions about my data. 

Suppose I'm curious about which cities in which my customers reside. I can make the query
```
SELECT id, name, city FROM customer
```
and my SQL engine will return the following table.

| id | name | city |
| --- | --- | --- |
| 1 | Maria Anders | Berlin | 
| 2 | Ana Trujillo | Mexico City |
| 3 | Antonio Moreno | Mexico City |
| 4 | Thomas Hardy | London |
| 5 | Christina Berglund | Luleå |

Now let's suppose my company is based in Mexico and I'm curious about the cities in which my *domestic* customers reside. I can make my query more specific with a `WHERE` clause
```
SELECT id, name, city FROM customer WHERE country = "Mexico"
```
to which my SQL engine returns

| id | name | city |
| --- | --- | --- |
| 2 | Ana Trujillo | Mexico City |
| 3 | Antonio Moreno | Mexico City |

> Aside: The true power of SQL becomes apparent when you start considering multiple tables. Suppose my company also has a database table storing all past orders.
>
> | id | customer_id | date |
> | --- | --- | --- |
> | 10308 |  2 | 1996-09-18 |
> | 10309 |  3 | 1996-09-19 |
> | 10310 |  4 | 1996-09-20 |
>
> I can use SQL to ask which of my orders came from Mexico and who made the orders. The query
> ```
> SELECT order.id, order.date, customer.name, customer.city
> FROM order JOIN customer ON order.customer_id = customer.id
> WHERE customer.country = "Mexico"
> ```
> returns the table
> 
> | order.id | order.date |  customer.name |  customer.city |
> | --- | --- | --- | --- |
> | 10308 | 1996-09-18 | Ana Trujillo | Mexico City |
> | 10309 | 1996-09-19 | Antonio Moreno | Mexico City |
>
> For now, we'll focus on SQL in the context of single tables.

### Exercise: extend our SQL grammar

Over the course of the next couple recitations, we'll develop a SQL engine that can run simple queries on your database. Before implementing the engine, which takes our queries and runs them, today we'll develop and extend the query language.

Here is the beginning of an implementation of our SQL grammar.

In [ ]:
sealed trait TableExpr
case class TableIdentifier(val s : String) extends TableExpr
case class SelectQuery(columns : List[Expr], table : TableExpr) extends TableExpr

sealed trait Expr
case class NumConst(n : Int) extends Expr
case class StringConst(s : String) extends Expr
case class ColumnIdentifier(val s : String) extends Expr

Our grammar so far allows us to express queries of the form
```
SELECT a, b, c FROM t
```
In our Scala grammar, this looks like
```
SelectQuery(List("a", "b", "c"), TableIdentifier("t"))
```
Essentially all we can express in our query language so far is to take existing tables and construct new tables that remove some of the columns. Your task is to extend our grammar with the `WHERE` clause, which allows the user to filter returned rows based on the values within some column. 
```
SELECT a, b, c FROM t WHERE a >= 1
```
The `WHERE` clause specifies a conditional expression, which can be the `true` or `false` constants, or otherwise make comparisons between `Expr` tokens. For a concrete example, consider above when we filtered down our customer records to those who reside in Mexico. 

Note that the `WHERE` clause is optional, so the user may choose to include it or not when defining their overall query.

In [ ]:
// modify at will
// you may need to add your own traits and/or case classes

sealed trait TableExpr
case class TableIdentifier(val s : String) extends TableExpr
case class SelectQuery(columns : List[Expr], table : TableExpr, where: WhereClause) extends TableExpr

// YOUR CODE HERE
sealed trait WhereClause
case class None() extends WhereClause
case class Where(c: CondExpr) extends WhereClause

sealed trait CondExpr
case class TrueConst() extends CondExpr
case class FalseConst() extends CondExpr
case class Eq(e1: Expr, e2: Expr) extends CondExpr

sealed trait Expr
case class NumConst(n : Int) extends Expr
case class StringConst(s : String) extends Expr
case class ColumnIdentifier(val s : String) extends Expr

> Extra: If you finish early, extend our query language with the `ORDER BY` clause.
> ```
> SELECT a, b, c FROM t ORDER BY a
> ```

## Pattern Matching

Implement a function, `isZigZag`, using pattern matching. The function should take a list of `Int`s and return `true` if the the numbers in the list switch from icreasing to decreasing to increasing to ... etc. at each position.

It should return `true` for the following inputs:
```
List()
List(1)
List(1, 2)
List(1, 2, 0, 1, -10, 10000, 9999)
```

It should return `false` for the following inputs:
```
List(1, 2, 3)
List(-1, -2, -3)
List(1, 2, 0, 1, -10, 10000, 9999, 9999)
```

In [27]:
// YOUR CODE HERE
def isZigZag(l: List[Int]): Boolean = {
    l match {
        case Nil => true
        case first :: Nil => true
        case first :: second :: Nil => first != second
        case first :: second :: third :: rest => {
            if (first < second && third < second) {isZigZag(second :: third :: rest)}
            else if (first > second && third > second) {isZigZag(second :: third :: rest)}
            else {false}
        }
        //Another way for last case
//         case first :: (rest @ second :: third :: _) => {
//             if (first < second && third < second) {isZigZag(rest)}
//             else if (first > second && third > second) {isZigZag(rest)}
//             else {false}
//         }
    }
}

defined function isZigZag

In [28]:
assert(isZigZag(List()), 1)
assert(isZigZag(List(1)), 2)
assert(isZigZag(List(1, 2)), 3)
assert(isZigZag(List(1, 2, 0, 1, -10, 10000, 9999)), 4)

assert(!isZigZag(List(1, 2, 3)), 5)
assert(!isZigZag(List(-1, -2, -3)), 6)
assert(!isZigZag(List(1, 2, 0, 1, -10, 10000, 9999, 9999)), 7)

## Higher order functions

Define a function called `map`. `map` should takes a list of `Int`s, and a function from `Int` to `Int`, then calls the function on each element and makes a new list from the results.

For example, the following call:
```
map(List(1, 2, 3, 4, 5), (x) => x + 1)
```
should return in the following list
```
List(2, 3, 4, 5, 6)
```

In [8]:
// YOUR CODE HERE
def map(l: List[Int], f: Int => Int): List[Int] = {
    l match {
        case Nil => Nil
        case first :: rest => f(first) :: map(rest, f)
    }
}

defined function map

In [9]:
assert(map(List(), _ + 1) == List())
assert(map(List(1), _ + 1) == List(2))
assert(map(List(1, 9, 2), _ + 1) == List(2, 10, 3))